In [47]:
from numpy import array
from collections import defaultdict
import pickle
from tqdm import tqdm
import scipy.cluster.hierarchy
words = []
WORDLENGTH = -1
with open("possibles.txt", "r") as file:
        for i in file:
            if len(i.strip()) == WORDLENGTH or WORDLENGTH == -1:
                words.append(i.strip())
calcs = {}
def freq(words):
    out = {}
    for i in words:
        for char, c in enumerate(i):
            if not c in out:
                out[c] = [0] * 5
            out[c][char] += 1
    return out
def calcWord(guess, word):
    global calcs
    if (guess, word) in calcs:
        return calcs[(guess, word)]
    counts = defaultdict(int)
    out = [0] * len(guess)
    for i, char in enumerate(word):
        counts[char] += 1
        if i < len(guess) and char == guess[i]:
            out[i] = "g"
            counts[char] -= 1
    for i in range(len(guess)):
        if out[i] == "g":
            continue
        elif counts[guess[i]] > 0:
            out[i] = "y"
            counts[guess[i]] -= 1
        elif type(guess) == str and guess[i].upper() in word:
            out[i] = "b"
        else:
            out[i] = "r"
    calcs[(guess, word)] = ''.join(out)
    return ''.join(out)
dist = {}
def distanceLetters(s1, s2):
    global pbar, frequency, dist
    out = 0
    i1, i2 = "", ""
    for c in range(5):
        i = s1[c*5:(c+1)*5]
        i1 += chr(ord('a')+list(i).index(max(i)))
    for c in range(5):   
        i = s2[c*5:(c+1)*5] 
        i2 += chr(ord('a')+ list(i).index(max(i)))
    letters = set(list(i1)+list(i2))
    for l in letters:
        for c in range(5):
            check = list("*****")
            check[c] = l
            check = ''.join(check)
            if calcWord(check, i1) != calcWord(check, i2):
                out += 1
    pbar.update(1)
    return out
def distanceWords(s1, s2):
    global pbar, frequency, dist
    if (tuple(s1), tuple(s2)) in dist:
        return dist[(tuple(s1), tuple(s2))]
    if (tuple(s2), tuple(s1)) in dist:
        return dist[(tuple(s2), tuple(s1))]
    out = 0
    i1 = ""
    i2 = ""
    print(i1, i2)
    for w in words:
            if calcWord(w, i1) != calcWord(w, i2):
                out += 1
    pbar.update(1)
    dist[(tuple(s1), tuple(s2))] = out
    dist[(tuple(s2), tuple(s1))] = out
    return out
def distanceVect(s1, s2):
    pbar.update(1)
    return sum(abs(s1-s2))
wordPoints = []
for word in words:
    temp1 = []
    for c in word:
        temp2 = [0] * 26
        temp2[ord(c) - ord('a')] = 1
        temp1 += temp2
    wordPoints.append(array(temp1))
#frequency = freq(wordPoints)
with tqdm(total=len(words)*(len(words)-1)/2, leave=False, position=0) as pbar:
    cluster = scipy.cluster.hierarchy.linkage(wordPoints, method="weighted", metric=distanceVect)
with open("cluster.p", "wb") as file:
    pickle.dump(cluster, file)

In [44]:
print(cluster[1])
print(words[12])
print(words[77])

[12. 77.  2.  2.]
abuse
amuse


In [48]:
import matplotlib.pyplot as plt
import itertools
%matplotlib widget
wordPoss = []
with open("cluster.p", "rb") as file:
    cluster = pickle.load(file)
with open("possibles.txt", "r") as file:
        for i in file:
            if len(i.strip()):
                wordPoss.append(i.strip())
newTree = []
labels = []
reps = []
for i in wordPoss:
    newTree.append([i])
    labels.append(i)
with open("cluster.p", "rb") as file:
    cluster = pickle.load(file)

plt.figure(figsize=(350, 20))
def test(i):
    return labels[i]
dend = scipy.cluster.hierarchy.dendrogram(cluster, labels=wordPoss, leaf_font_size=7, leaf_label_func=test)
ax = plt.gca()
ax.get_yaxis().set_visible(False)
plt.savefig('plt.jpg', bbox_inches='tight')

In [46]:
import itertools
from numpy import square
def getBestThree(wordListGuess, wordListPoss, calcFunc):
  out = defaultdict(int)
  if not isinstance(wordListPoss[0], list):
      wordListPoss = [wordListPoss]
  for c, guess in enumerate(wordListGuess):
    for wordlist in wordListPoss:
        contain = defaultdict(int)
        for poss in wordlist:
            if guess != poss:
                contain[calcFunc(guess, poss)] += 1
        out[guess] += sum(square(array(list(contain.values())))/len(wordlist))/len(wordListPoss)
  wordFreqSort = dict(sorted(out.items(), key=lambda item: item[1]))
  return wordFreqSort
def rep(words):
    output = list("*****")
    did = []
    wordsNew = [w for w in words if w != ""]
    for i in range(5):
        same = True
        for word in wordsNew:
            if word[i] != wordsNew[0][i]:
                same = False
                break
        if same:
            output[i] = wordsNew[0][i]
            did.append(output[i])
    for i in "abcdefghijklmnopqrstuvwxyz":
        if i in did:
            continue
        inAll = True
        inNone = True
        for word in wordsNew:
            if not i in word or i.upper() in word:
                inAll = False
            if i in word or not i.upper() in word:
                inNone = False
            if not inAll and not inNone:
                break
        if inAll:
            output += i
        if inNone:
            output += i.upper()
    return ''.join(output)
wordPoss = []
with open("possibles.txt", "r") as file:
        for i in file:
            if len(i.strip()) == WORDLENGTH or WORDLENGTH == -1:
                wordPoss.append(i.strip())
newTree = []
for i in wordPoss:
    newTree.append([i])
with open("cluster.p", "rb") as file:
    cluster = pickle.load(file)

for i in tqdm(cluster, leave = False, position = 0):
    newTree.append(list(itertools.chain(newTree[int(i[0])], newTree[int(i[1])])))
print(newTree[-1])

['yield', 'field', 'wield', 'elide', 'glide', 'ideal', 'idler', 'plied', 'lipid', 'plaid', 'solid', 'valid', 'livid', 'vivid', 'build', 'built', 'guild', 'guilt', 'quill', 'quilt', 'pupil', 'tulip', 'clued', 'cloud', 'could', 'druid', 'lurid', 'fluid', 'lucid', 'gland', 'clang', 'slang', 'alone', 'along', 'glean', 'angel', 'angle', 'nylon', 'salon', 'talon', 'zonal', 'banal', 'nasal', 'natal', 'equal', 'usual', 'mural', 'rural', 'annul', 'lunar', 'relax', 'learn', 'renal', 'melee', 'medal', 'metal', 'realm', 'rearm', 'alarm', 'llama', 'plain', 'plait', 'pizza', 'plaza', 'plank', 'plant', 'alpha', 'papal', 'elate', 'plate', 'leapt', 'petal', 'pleat', 'ample', 'maple', 'lapse', 'apple', 'lapel', 'pedal', 'plead', 'plane', 'panel', 'penal', 'final', 'flail', 'flair', 'frail', 'flaky', 'flask', 'fecal', 'fatal', 'fetal', 'feral', 'flare', 'flake', 'flame', 'false', 'fable', 'fella', 'afoot', 'aloft', 'float', 'afoul', 'offal', 'awful', 'aglow', 'allow', 'pouty', 'putty', 'empty', 'petty', 

In [39]:
for n in newTree:
    print(n)

['abackDEFGHIJLMNOPQRSTUVWXYZ']
['abaseCDFGHIJKLMNOPQRTUVWXYZ']
['abateCDFGHIJKLMNOPQRSUVWXYZ']
['abbeyCDFGHIJKLMNOPQRSTUVWXZ']
['abbotCDEFGHIJKLMNPQRSUVWXYZ']
['abhorCDEFGIJKLMNPQSTUVWXYZ']
['abideCFGHJKLMNOPQRSTUVWXYZ']
['abledCFGHIJKMNOPQRSTUVWXYZ']
['abodeCFGHIJKLMNPQRSTUVWXYZ']
['abortCDEFGHIJKLMNPQSUVWXYZ']
['aboutCDEFGHIJKLMNPQRSVWXYZ']
['aboveCDFGHIJKLMNPQRSTUWXYZ']
['abuseCDFGHIJKLMNOPQRTVWXYZ']
['abyssCDEFGHIJKLMNOPQRTUVWXZ']
['acornBDEFGHIJKLMPQSTUVWXYZ']
['acridBEFGHJKLMNOPQSTUVWXYZ']
['actorBDEFGHIJKLMNPQSUVWXYZ']
['acuteBDFGHIJKLMNOPQRSVWXYZ']
['adageBCFHIJKLMNOPQRSTUVWXYZ']
['adaptBCEFGHIJKLMNOQRSUVWXYZ']
['adeptBCFGHIJKLMNOQRSUVWXYZ']
['adminBCEFGHJKLOPQRSTUVWXYZ']
['admitBCEFGHJKLNOPQRSUVWXYZ']
['adobeCFGHIJKLMNPQRSTUVWXYZ']
['adoptBCEFGHIJKLMNQRSUVWXYZ']
['adoreBCFGHIJKLMNPQSTUVWXYZ']
['adornBCEFGHIJKLMPQSTUVWXYZ']
['adultBCEFGHIJKMNOPQRSVWXYZ']
['affixBCDEGHJKLMNOPQRSTUVWYZ']
['afireBCDGHJKLMNOPQSTUVWXYZ']
['afootBCDEGHIJKLMNPQRSUVWXYZ']
['afoulBCDEGHIJKMNPQRSTVWXYZ'